# 0. Import Packages

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import os
import glob
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import math

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras_preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Dropout
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import model_from_json
from keras import optimizers

# 1. Load data

In [55]:
# 온도 선택 [N10, 0, 10, 20, 25, 30, 40, 50], temp 변수에 설정

temp = 'N10'
mode = ['DST', 'FUDS' ,'US06']
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

for m in mode:
    data_dir = os.path.join(parent_dir, f'DB Preprocessing/refined_data/{temp}/{m}')
    file_names = os.listdir(data_dir)
    for file_name in file_names:
        csv_dir = os.path.join(data_dir, file_name)
        if '007' in file_name:
            num = '007'
        else: 
            num = '008'
        globals()['csv_{}'.format(f'{m}_{num}')] = pd.DataFrame(pd.read_csv(csv_dir))
        print(f'csv_{m}_{num} 생성 완료')

csv_DST_007 생성 완료
csv_DST_008 생성 완료
csv_FUDS_007 생성 완료
csv_FUDS_008 생성 완료
csv_US06_007 생성 완료
csv_US06_008 생성 완료


In [56]:
csv_DST_007

,Current(A),Voltage(V),Temperature (C)_1,SoC
0,0.000191,3.526907,-8.332584,80.000000
1,0.000191,3.526599,-8.565621,80.000000
2,0.000191,3.526291,-8.332584,80.000000
3,0.000003,3.525676,-8.332584,80.000000
4,0.000378,3.525984,-8.332584,80.000000
...,...,...,...,...
5279,-0.480845,2.945211,-7.249700,0.356594
5280,-3.849219,2.086517,-7.094693,0.275559
5281,-3.849219,2.045275,-7.094693,0.176988
5282,-3.849219,2.017267,-6.985489,0.079046


# 2. V, I, T / SoC 분리 

In [57]:
num = ['007','008']

def df2numpy(df):
    np = df.values
    return np

for m in mode:
    for n in num:
        var_name = f'csv_{m}_{n}'
        csv = globals()[var_name]
        globals()[f'input_{m}_{n}'] = df2numpy(csv[['Current(A)', 'Voltage(V)', 'Temperature (C)_1']])
        print(f'input_{m}_{n} 생성 완료')
        globals()[f'output_{m}_{n}'] = df2numpy(csv[['SoC']])
        print(f'output_{m}_{n} 생성 완료')

input_DST_007 생성 완료
output_DST_007 생성 완료
input_DST_008 생성 완료
output_DST_008 생성 완료
input_FUDS_007 생성 완료
output_FUDS_007 생성 완료
input_FUDS_008 생성 완료
output_FUDS_008 생성 완료
input_US06_007 생성 완료
output_US06_007 생성 완료
input_US06_008 생성 완료
output_US06_008 생성 완료


In [58]:
print(input_DST_007.shape)
input_DST_007

(5284, 3)


array([[ 1.90588195e-04,  3.52690697e+00, -8.33258438e+00],
       [ 1.90588195e-04,  3.52659917e+00, -8.56562138e+00],
       [ 1.90588195e-04,  3.52629137e+00, -8.33258438e+00],
       ...,
       [-3.84921932e+00,  2.04527473e+00, -7.09469271e+00],
       [-3.84921932e+00,  2.01726723e+00, -6.98548889e+00],
       [-3.84921932e+00,  1.99972391e+00, -7.09469271e+00]])

In [59]:
print(output_DST_007.shape)
output_DST_007

(5284, 1)


array([[8.00000000e+01],
       [8.00000000e+01],
       [8.00000000e+01],
       ...,
       [1.76987717e-01],
       [7.90458071e-02],
       [0.00000000e+00]])

# 3. Split train and test dataset

In [69]:
def create_dataset(input, output, look_back=1):
    dataX, dataY = [], []
    for i in range(len(input) - look_back + 1):
        a = input[i:(i + look_back)]
        dataX.append(a)
        dataY.append(output[i + look_back -1])
    dataY = np.reshape(dataY, (len(dataY), 1, 1))
    print(dataY.shape)
    return np.array(dataX), np.array(dataY)

a = [0,1,2,3,4,5,6,7,8,9]
b = [10,11,12,13,14,15,16,17,18,19]
x, y = create_dataset(a, b, 3)
print(x)
print(y)

(8, 1, 1)
[[0 1 2]
 [1 2 3]
 [2 3 4]
 [3 4 5]
 [4 5 6]
 [5 6 7]
 [6 7 8]
 [7 8 9]]
[[[12]]

 [[13]]

 [[14]]

 [[15]]

 [[16]]

 [[17]]

 [[18]]

 [[19]]]


In [70]:
# look_back 변수 설정(자연수. 보통 20 설정.)
look_back = 20
dstX, dstY = create_dataset(input_DST_007, output_DST_007, look_back)
us06X, us06Y = create_dataset(input_US06_007, output_DST_007, look_back)
fudsX, fudsY = create_dataset(input_FUDS_007, output_FUDS_007, look_back)

(5265, 1, 1)
(5651, 1, 1)
(5265, 20, 3)
(5265, 1, 1)
